# Research Question 3

## Can we assess the quality of generated commit messages by comparing their style?

https://arxiv.org/pdf/2006.00843.pdf

In [1]:
import torch
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.metrics.pairwise import euclidean_distances

import warnings
warnings.filterwarnings('ignore')

import sys
sys.path.append("..")
from util.style_model import StyleModel

In [2]:
test_data = pd.read_pickle('../data/04-1c_Authors_Test_Set.pkl')

In [3]:
model = StyleModel()
model.load_state_dict(torch.load('../model/Authors_StyleModel.pt'))

<All keys matched successfully>

In [4]:
messages = test_data["message"].tolist()

vectors = []

for message in tqdm(messages):
    vectors.append(model(message).squeeze().detach().numpy())

vectors = np.array(vectors)

100%|██████████| 14536/14536 [05:43<00:00, 42.26it/s]


In [5]:
authors_centroids = {}

for group in test_data.groupby('author_email'):
    author_embeddings = np.array(vectors[group[1].index])
    authors_centroids[group[0]] = np.mean(author_embeddings, axis = 0)

In [6]:
centroids_array = [value for value in authors_centroids.values()]

## Self-written Messages

In [7]:
good_message_embedding = model("MINOR Removed unused jQuery.dialog creation in CMSMain.AddFor.js, which causes mem leaks (now uses dedicated pages/add UI) ").detach().numpy()
bad_message_embedding = model("Update files").detach().numpy()
worst_message_embedding = model("12345").detach().numpy()

In [8]:
distances = pd.DataFrame()

distances["Good Message: \"MINOR Removed unused ...\""] = pd.DataFrame(euclidean_distances(centroids_array, good_message_embedding))
distances["Bad Message: \"Update files\""] = pd.DataFrame(euclidean_distances(centroids_array, bad_message_embedding))
distances["Worst Message: \"12345\""] = pd.DataFrame(euclidean_distances(centroids_array, worst_message_embedding))

distances.index = ['Author 1', 'Author 2', 'Author 3', 'Author 4', 'Author 5', 'Author 6', 'Author 7', 'Author 8', 'Author 9', 'Author 10', 'Author 11', 'Author 12', 'Author 13', 'Author 14', 'Author 15']

distances.style.background_gradient(cmap='coolwarm', axis=None).set_precision(3)

,"Good Message: ""MINOR Removed unused ...""","Bad Message: ""Update files""","Worst Message: ""12345"""
Author 1,0.820,1.071,1.642
Author 2,0.588,1.157,1.738
Author 3,0.625,1.070,1.724
Author 4,0.638,1.080,1.677
Author 5,0.613,1.106,1.771
Author 6,0.631,1.049,1.686
Author 7,0.626,1.140,1.747
Author 8,0.607,1.090,1.728
Author 9,0.676,0.952,1.702
Author 10,0.621,1.005,1.719


## Messages from the dataset

In [21]:
bad_messages = []

for i, vector in enumerate(vectors):
    if np.mean(euclidean_distances(centroids_array, vector.reshape(1, -1))) > 1:
        bad_messages.append(messages[i])

print("\n\nDetected", len(bad_messages), "messages of bad quality out of a total of", len(vectors), "messages:\n")
print(*bad_messages, sep="\n\n")



Detected 2315 messages of bad quality out of a total of 14536 messages:

[enrich][logs] Show datasource in loading identities logs to identify logs
when working with several data sources in different threads at the same time.

[enrich][bugzilla] Use items as the type for the ES index

[track_items] Check that upstream contributors file exists. If not, log it and return and
empty list of items.

[enrich][supybot] Add project support

[enrich][git] Manage not existing commits in github when getting author info

[Bugzilla SH Identities] Add email if we have this data.

[arthur] Remove the raw items from redis queue once they are processed

[config] Fix howto import the version from mordred

[enrich][jira] Include release field in enriched index

[ocean] Fix getting data from ocean using offset for incremental enrich

Changed default self.max_items_bulk to <I> so it works in remote elastic cases.

[confluence][enrich] Support that username or displayName are not included in user data

[e

In [10]:
# do the same for good messages
good_messages = []

for i, vector in enumerate(vectors):
    if np.mean(euclidean_distances(centroids_array, vector.reshape(1, -1))) < 0.7:
        good_messages.append(messages[i])

print("\n\nDetected", len(bad_messages), "messages of bad quality out of a total of", len(vectors), "messages:\n")
print(*good_messages, sep="\n\n")



Detected 32 messages of bad quality out of a total of 14536 messages:

[release] Update version number to <I>

[logs] Remove logs related to getting last update.

[release] Update version number to <I>

[tests] Remove not needed imports

[task_collection] Add a specific tag param to arthur collected raw items

[enrich][twitter] Check that twitter is in projects map before using it

Before trying to get the map between a hashtag and a projects,
check that twitter is included in the projects map.

[gh2arthur] Use "scm" and data source for git in projects table.

[enrich] Support in safe_index to receive None as the name of the index

[release] Update version number to <I>

[release] Update version number to <I>

[enrich][discourse] The category_id must be a string in the projects mapping

[release] Update version number to <I>

[release] Update version number to <I>

[config] Add a set_param method to change a config param

[release] Update version number to <I>

Increase version packa